## Required Imports

In [0]:
!pip3 install pyro-ppl

import pyro
import torch
import torch.nn
import matplotlib
import pyro.optim as optim
import pyro.distributions as dist
from torch.distributions import constraints, Normal, Categorical
import seaborn as sns
from pyro.infer import SVI, Trace_ELBO
from tqdm import tqdm_notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython import display
import os
from PIL import Image
from torch.utils.data.dataset import Dataset
# from scipy.misc import imread
%matplotlib inline

     |████████████████████████████████| 235kB 4.8MB/s 
     |████████████████████████████████| 61kB 20.3MB/s 
     |████████████████████████████████| 51kB 17.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/37/6b/8b/8d15c6042ed38db155158baf56c1949a6e12d5d709697b0c37
  Stored in directory: /root/.cache/pip/wheels/51/3e/a3/b351fae0cbf15373c2136a54a70f43fea5fe91d8168a5faaa4
Successfully built pyro-ppl opt-einsum
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [0]:
df = sns.load_dataset('iris')
df.loc[df['species'].values == 'setosa', df.columns == 'species'] = 0
df.loc[df['species'].values == 'virginica', df.columns == 'species'] = 1
df.loc[df['species'].values == 'versicolor', df.columns == 'species'] = 2
df.head()
x = torch.Tensor(df.loc[:, df.columns != 'species'].values)
y = torch.Tensor(df.loc[:, df.columns == 'species'].values)
y = y.reshape((-1, ))

## Naive Bayes

In [0]:
data = datasets.load_iris()
x_data = torch.Tensor(data.data)
y_data = torch.Tensor(data.target)
samples_count = len(y_data)
C_count = 3
A_count = 4

In [0]:
def calc_class_probs(x, mean, std):
    class_probs = torch.ones(samples_count, C_count)
    
    for c in range(C_count):
      
      att_probs = torch.ones(samples_count, A_count)
      for a in range(A_count):
        value = Normal(loc=mean[a, c], scale=softplus(std[a, c])).log_prob(x[:, a])
        att_probs[:,a] = value
      
      sum = torch.sum(att_probs, dim=1)
      class_probs[:,c] = sum
    
    class_probs = torch.exp(class_probs)
    
    return class_probs


softplus = torch.nn.Softplus()

In [0]:
def model(x_data, y_data):
  
  with pyro.plate('c', C_count):
    with pyro.plate('a', A_count):
      mean_prior = pyro.distributions.Normal(loc=torch.ones(A_count, C_count), scale=torch.ones(A_count, C_count))
      std_prior = pyro.distributions.Normal(loc=torch.ones(A_count, C_count), scale=torch.ones(A_count, C_count))
      mean_sample = pyro.sample('mean_sample', mean_prior)
      std_sample = pyro.sample('std_sample', std_prior)
      
  class_probs = calc_class_probs(x_data, mean_sample, std_sample)
  
  with pyro.plate('map', len(y_data)):
    predictions = Categorical(probs=class_probs)
    pyro.sample("obs", predictions, obs=y_data)

In [0]:
def guide(x_data, y_data):
    mean_mean = pyro.param("mean_mean", torch.ones(A_count, C_count))
    mean_scale = pyro.param("mean_std", torch.ones(A_count, C_count), constraint=constraints.positive)
    std_mean = pyro.param("std_mean", torch.ones(A_count, C_count))
    std_scale = pyro.param("std_std", torch.ones(A_count, C_count), constraint=constraints.positive)
    
 
    with pyro.plate('c', size=C_count):
        with pyro.plate('a', size=A_count):
            mean_prior = pyro.distributions.Normal(loc=mean_mean, scale=mean_scale)
            std_prior = pyro.distributions.Normal(loc=std_mean, scale=std_scale)
            mean_sample = pyro.sample('mean_sample', mean_prior)
            std_sample = pyro.sample('std_sample', std_prior)

            
    return mean_sample, std_sample

In [0]:
def train():
    pyro.clear_param_store()
    num_iterations=1000
    optim = pyro.optim.Adam({"lr": 0.05})
    svi = SVI(model, guide, optim, loss=Trace_ELBO())
    t=tqdm(range(num_iterations))
    for j in t:
        loss = svi.step(x, y)
        t.set_postfix(loss=loss)
    return (model, svi)

In [0]:
train()

params = guide(None, None)

class_probs = calc_class_probs(x, params[0], params[1])
predictions = torch.argmax(class_probs, dim=1)

accuracy = np.count_nonzero((y==predictions.type(torch.float)).detach().numpy()) / len(predictions)
print(accuracy)

0.96


In [0]:
NaiveBayes = GaussianNB()
NaiveBayes.fit(x_data, y_data)
y_hat = NaiveBayes.predict(x_data)
metrics.accuracy_score(y_data, y_hat)

0.96